<a href="https://colab.research.google.com/github/nixmaverick1997/CNNTextClassificationwithGloVEEmbeddings/blob/main/NLP_spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# libraries  
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import re
import unicodedata
from zipfile import ZipFile
from google.colab import files
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout, Activation,GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras import Input,Model
import keras


nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Dataset

This corpus has been collected from free or free for research sources at the Internet: 
* A collection of 425 SMS spam messages was manually extracted from the Grumbletext Web site. This is a UK forum in which cell phone users make public claims about SMS spam messages, most of them without reporting the very spam message received. The identification of the text of spam messages in the claims is a very hard and time-consuming task, and it involved carefully scanning hundreds of web pages. The Grumbletext Web site is: http://www.grumbletext.co.uk/. 
* A subset of 3,375 SMS randomly chosen ham messages of the NUS SMS Corpus (NSC), which is a dataset of about 10,000 legitimate messages collected for research at the Department of Computer Science at the National University of Singapore. The messages largely originate from Singaporeans and mostly from students attending the University. These messages were collected from volunteers who were made aware that their contributions were going to be made publicly available.

In [ ]:
# import dataset

files.upload()

with ZipFile('smsspamcollection.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

df = pd.read_csv('SMSSpamCollection',sep="\t",header=None)
#corpus = []
#wordnet = WordNetLemmatizer()
#length = len(df['v2']) #nb rows
df=df.rename(columns={0: "Class", 1: "SMS_Content"})
df

Saving smsspamcollection.zip to smsspamcollection (2).zip


,Class,SMS_Content
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


# Pre-processing input data corpuses

In [ ]:
# pre-processing
# sms raw text need to be "normalized" i.e. whole text in lowercase, no stopwords, no punctuation, etc
df['SMS_Content_norm']=df['SMS_Content'].str.lower()

def remove_whitespace(text):
    return  " ".join(text.split())
df['SMS_Content_norm']=df['SMS_Content_norm'].apply(remove_whitespace)

def remove_accented_chars(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text
df['SMS_Content_norm']=df['SMS_Content_norm'].apply(remove_accented_chars)

df

,Class,SMS_Content,SMS_Content_norm
0,ham,"Go until jurong point, crazy.. Available only ...","go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah i don't think he goes to usf, he lives aro..."
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,this is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?,will u b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s...","pity, * was in mood for that. so...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...,the guy did some bitching but i acted like i'd...


In [ ]:
def text_to_wordlist(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"U ", "you ", text)
    text = re.sub(r" u ", " you ", text)
    text = re.sub(r"ü ", "you ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"da ", " the ", text) 
    text = re.sub(r" b ", " be ", text)
    text = re.sub(r"'m", " am ", text)
    text = re.sub(r" IM ", " i am ", text)
    text = re.sub(r" im ", " i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r" wd ", " would ", text)
    text = re.sub(r" shd ", " should ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text

df['SMS_Content_norm']=df['SMS_Content_norm'].apply(text_to_wordlist)
df['SMS_Content_norm']=df['SMS_Content_norm'].apply(remove_whitespace)


df

,Class,SMS_Content,SMS_Content_norm
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis great world...
1,ham,Ok lar... Joking wif u oni...,lar joking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win cup final tkts 21st m...
3,ham,U dun say so early hor... U c already then say...,dun say early hor already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think goes usf lives around though
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,2nd time tried contact 750 pound prize claim e...
5568,ham,Will ü b going to esplanade fr home?,going esplanade home
5569,ham,"Pity, * was in mood for that. So...any other s...",pity mood that so any suggestions
5570,ham,The guy did some bitching but I acted like i'd...,guy bitching acted like i would interested buy...


In [ ]:
#convert text to integer sequences

from keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=20)

vectorizer.adapt(df['SMS_Content_norm'])

In [ ]:
#forming vocabulary from the source dataset
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))
vocab_size=len(word_index)

In [ ]:
docs=df['SMS_Content_norm'].tolist()
#sequences = tokenizer.texts_to_sequences(docs)

padded_data = vectorizer(docs)
#pad_sequences(sequences, maxlen=20,padding='post')

padded_data[0]

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([6400,  739,  681,  576, 1336,   55,  242, 3918, 1331,   14, 8059,
         63,    0,    0,    0,    0,    0,    0,    0,    0])>

Label encoding the class

In [ ]:
le = LabelEncoder()
le.fit(df["Class"])
target=le.transform(df["Class"])

# Downloading GloVe Embedding matrix

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2022-06-15 22:43:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-06-15 22:43:27--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-06-15 22:43:28--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

# Creating the CNN model for classification (a simple conv1dnet)

In [ ]:
import os

path_to_glove_file = os.path.join("glove.6B.100d.txt")

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Found 400000 word vectors.
Converted 6459 words (2170 misses)


In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))

In [ ]:
for word, index in word_index.items():
    if index > vocab_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [ ]:
print(embedding_matrix.shape)

(8629, 100)


In [ ]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = Conv1D(128, 2, activation="relu")(embedded_sequences)
x = MaxPooling1D(2)(x)
x = Conv1D(128, 2, activation="relu")(x)
x = MaxPooling1D(2)(x)
x = Conv1D(128, 2, activation="relu")(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
preds = Dense(2, activation="softmax")(x)
model = Model(int_sequences_input, preds)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_8 (Embedding)     (None, None, 100)         863100    
                                                                 
 conv1d_24 (Conv1D)          (None, None, 128)         25728     
                                                                 
 max_pooling1d_20 (MaxPoolin  (None, None, 128)        0         
 g1D)                                                            
                                                                 
 conv1d_25 (Conv1D)          (None, None, 128)         32896     
                                                                 
 max_pooling1d_21 (MaxPoolin  (None, None, 128)        0         
 g1D)                                                      

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df["SMS_Content_norm"], target, test_size=0.2, random_state=42)

X_train = vectorizer(np.array([[s] for s in X_train])).numpy()
X_test = vectorizer(np.array([[s] for s in X_test])).numpy()

y_train = np.array(y_train)
y_test = np.array(y_test)
X_train[0]

array([  38,   98,  228,  420, 8391, 6992,  242, 1061, 3586,   20,   32,
       2138,  223,  136,    0,    0,    0,    0,    0,    0])

In [ ]:
loss_function = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(
    loss=loss_function, optimizer="adam", metrics=["acc"]
)

model.fit(X_train, y_train, epochs=23, verbose=1)
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Epoch 1/23


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


140/140 [==============================] - 2s 7ms/step - loss: 0.1850 - acc: 0.9349
Epoch 2/23
140/140 [==============================] - 1s 7ms/step - loss: 0.0698 - acc: 0.9789
Epoch 3/23
140/140 [==============================] - 1s 7ms/step - loss: 0.0388 - acc: 0.9888
Epoch 4/23
140/140 [==============================] - 1s 7ms/step - loss: 0.0177 - acc: 0.9951
Epoch 5/23
140/140 [==============================] - 1s 7ms/step - loss: 0.0229 - acc: 0.9926
Epoch 6/23
140/140 [==============================] - 1s 7ms/step - loss: 0.0092 - acc: 0.9973
Epoch 7/23
140/140 [==============================] - 1s 7ms/step - loss: 0.0027 - acc: 0.9996
Epoch 8/23
140/140 [==============================] - 1s 7ms/step - loss: 0.0014 - acc: 0.9998
Epoch 9/23
140/140 [==============================] - 1s 7ms/step - loss: 0.0012 - acc: 0.9998
Epoch 10/23
140/140 [==============================] - 1s 7ms/step - loss: 0.0013 - acc: 0.9998
Epoch 11/23
140/140 [==============================] - 1s 7m

In [ ]:
from keras import Input, Model
#model.predict([[text_predict]])
string_input = Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["going try months joking"]]
)

In [ ]:
y_predict  = [1 if o>0.5 else 0 for o in probabilities[0]]
y_predict
probabilities[0]

array([1.000000e+00, 2.909413e-11], dtype=float32)

In [ ]:
le.inverse_transform(y_predict)[np.argmax(y_predict)]

'spam'

# Creating word2vec vectors from GloVe embedding

In [ ]:
from gensim.models import KeyedVectors

w2v = KeyedVectors.load_word2vec_format('glove.6B.300d.word2vec.txt',binary=False)

In [ ]:
# King + Woman - Man = ?
w2v.most_similar(['king','woman'],negative=['man'],topn=1)

[('queen', 0.6713277101516724)]

In [ ]:
w2v.most_similar('king')

[('queen', 0.6336469054222107),
 ('prince', 0.619662344455719),
 ('monarch', 0.5899620652198792),
 ('kingdom', 0.5791267156600952),
 ('throne', 0.5606487989425659),
 ('ii', 0.5562329888343811),
 ('iii', 0.5503199100494385),
 ('crown', 0.5224862694740295),
 ('reign', 0.521735429763794),
 ('kings', 0.5066401362419128)]

In [ ]:
w2v.doesnt_match("france venezuela mexico brazil".split())

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'france'